<a href="https://colab.research.google.com/github/Jessvcv/Statistical-Modeling-HW/blob/main/Tran_HW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ISLP

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 5.0 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=6144507bd1683a7292e651558a9f9a87a1c26210f9997d04db31cd2457dbf13d
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from ISLP import confusion_table
from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.metrics import accuracy_score
from ISLP.models import sklearn_sm
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.preprocessing import scale
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


# Pima

In [ ]:
pima_url = 'https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/master/diabetes.csv'

In [ ]:
pima_df = pd.read_csv(pima_url)

In [ ]:
pima_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


# Q3. (2 points)
Build an ordinary least squares model which uses all of the features to predict BMI.

*   0.5 points : build X and Y
*   0.5 points : scale X
*   0.5 points : train test split where 30% of data is reserved for testing
* 0.5 points : build OLS model and print test set MSE.





In [ ]:
# Build X & Y
X = pima_df.drop(columns=['BMI'])
Y = pima_df['BMI']

# Scale X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train test split where 30% of data is reserved for testing
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y, test_size=0.3)

# OLS Model:

# Add constant for intercept
X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

# Fit OLS model
ols_model = sm.OLS(Y_train, X_train_sm).fit()

# Predict on test data
Y_pred = ols_model.predict(X_test_sm)

# Calculate & Print test MSE
test_mse = mean_squared_error(Y_test, Y_pred)

print("Test set MSE:", test_mse)



Test set MSE: 47.5293001851732


# Q4. (1 point)
Now evaluate your model using k-fold cross validation (k=5) and LOOCV. print both MSE values.

In [ ]:
# Define model
lr = LinearRegression()

# 5-Fold Cross Validation
kfold = KFold(n_splits=5, shuffle=True)
kfold_mse = cross_validate(
    lr, X_scaled, Y, cv=kfold, scoring='neg_mean_squared_error'
)['test_score']

# Leave-One-Out Cross Validation (LOOCV)
loo = LeaveOneOut()
loo_mse = cross_validate(
    lr, X_scaled, Y, cv=loo, scoring='neg_mean_squared_error'
)['test_score']

# Convert from negative MSE to positive
kfold_mse_mean = -np.mean(kfold_mse)
loo_mse_mean = -np.mean(loo_mse)

# Print results
print(f"5-Fold CV MSE: {kfold_mse_mean:.4f}")
print(f"LOOCV MSE: {loo_mse_mean:.4f}")

5-Fold CV MSE: 46.9725
LOOCV MSE: 46.8664


# Q8. (1 point)
 Build model with Ridge using RidgeCV to predict BMI. Report your optimal lambda (alpha, tuning parameter) value and your best MSE.

In [ ]:
# Define alpha (lambda) values to test
alphas = np.logspace(-3, 3, 100)

# Create and fit RidgeCV model (built-in cross-validation)
ridge_cv = RidgeCV(alphas=alphas, store_cv_values=True)
ridge_cv.fit(X_scaled, Y)

# Predict using RidgeCV model
Y_pred_ridge = ridge_cv.predict(X_scaled)

# Compute MSE
ridge_mse = mean_squared_error(Y, Y_pred_ridge)

# Print optimal lambda and MSE
print(f"Optimal lambda (alpha): {ridge_cv.alpha_:.4f}")
print(f"Best Ridge MSE: {ridge_mse:.4f}")

Optimal lambda (alpha): 46.4159
Best Ridge MSE: 45.4824


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_ridge.py:2385: FutureWarning: 'store_cv_values' is deprecated in version 1.5 and will be removed in 1.7. Use 'store_cv_results' instead.
  warnings.warn(


# Q9. (1 point)
Build model with Lasso to predict BMI. Report your optimal  lambda (alpha, tuning parameter) value and your coefficients. Which are eliminated by your model?

In [ ]:
# Define alpha (lambda) values to test
alphas = np.logspace(-3, 3, 100)

# Build and fit LassoCV model
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=42)
lasso_cv.fit(X_scaled, Y)

# Get optimal lambda and coefficients
best_alpha = lasso_cv.alpha_
coefficients = lasso_cv.coef_

# Print results
print(f"Optimal lambda (alpha): {best_alpha:.4f}")
print("Lasso Coefficients:")
for name, coef in zip(pima_df.drop(columns=['BMI']).columns, coefficients):
    print(f"{name:20s}: {coef:.4f}")

# Identify eliminated (zero) coefficients
eliminated = [name for name, coef in zip(pima_df.drop(columns=['BMI']).columns, coefficients) if coef == 0]
print("\nEliminated features:", eliminated)

Optimal lambda (alpha): 0.0433
Lasso Coefficients:
Pregnancies         : -0.1655
Glucose             : 0.5967
BloodPressure       : 1.5292
SkinThickness       : 2.5500
Insulin             : -0.1313
DiabetesPedigreeFunction: 0.1688
Age                 : -0.2490
Outcome             : 1.7782

Eliminated features: []
